In [1]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import model_afolu as ma
import model_circular_economy as mc
from model_circular_economy import CircularEconomy
from model_socioeconomic import Socioeconomic
import model_ippu as mi
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(mi)



/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:637: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:637: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_ippu' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_ippu.py'>

In [275]:
df_tmp = sa.model_attributes.transfer_df_variables(
    df_complete,
    df_project_afolu,
    model_circecon.integration_variables
)

In [277]:
os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv")

'/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/fake_data/fake_data_complete.csv'

In [2]:
df_complete = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_waste = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_circular_economy.csv"))
df_socio = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_socioeconomic.csv"))
#model_afolu = ma.AFOLU(sa.model_attributes);
model_circecon = mc.CircularEconomy(sa.model_attributes);
#df_project_afolu = model_afolu.project(df_complete)
# pass integrated df
#df_waste_integrated = df_complete#pd.merge(df_complete, df_project_afolu)
df_project_ce = model_circecon.project(df_complete);
df_waste_integrated = sf.merge_output_df_list([df_complete, df_project_ce], sa.model_attributes, "concatenate")



In [235]:
df_vals = pd.read_excel(sa.excel_template_path("IPPU", "arg", "demo"))
df_calib_info = pd.read_excel("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/preliminary_calibration_info/df_ip_var_calib.xlsx", skiprows = 2)

vars_input = sa.model_attributes.build_varlist("IPPU", variable_type = "input")
df_calib_info = df_calib_info[df_calib_info["Variable"].isin(vars_input)]
vars_new = list(set(vars_input) - set(df_calib_info["Variable"]))
df_copy = df_vals[df_vals["variable"].isin(vars_new)]
df_copy.to_csv("/Users/jsyme/Desktop/tmp2.csv", index = None, encoding = "UTF-8")
df_calib_info.to_csv("/Users/jsyme/Desktop/tmp_keep.csv", index = None, encoding = "UTF-8")



In [3]:
importlib.reload(ds)
importlib.reload(sa)
importlib.reload(mi)
model_ippu = mi.IPPU(sa.model_attributes)
df_ippu = model_ippu.project(df_waste_integrated)
df_ippu[[x for x in df_ippu.columns if "cement" in x]].tail()

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:637: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


,emission_co2e_co2_ippu_production_cement,prod_ippu_cement_tonne
31,0.052236,106267.076244
32,0.052342,106481.180521
33,0.052448,106695.716566
34,0.052554,106910.685017
35,0.052660,107126.086504


In [161]:
#np.cumprod(np.arange(1,5)/4.3)
#for k in df_ippu.columns:
#    print(k)
#df_ippu0 = df_ippu.copy();
attr_ippu.get_key_value_index("glass")
dict1 = {1: 4, 8: 10}
[dict1.get(x, x) for x in [8, 1, 9]]
df_ippu[[x for x in df_ippu.columns if "cement" in x]].tail()

[10, 4, 9]

In [209]:
df_ippu[[x for x in df_ippu.columns if "cement" in x]]

,emission_co2e_co2_ippu_production_cement,prod_ippu_cement_tonne
0,0.049140,100000.000000
1,0.049259,100241.798454
2,0.049355,100435.727203
3,0.049430,100586.379616
4,0.049512,100753.877774
5,0.049592,100915.464301
6,0.049678,101088.747026
7,0.049766,101268.144758
8,0.049862,101460.549855
9,0.049962,101664.879177


In [139]:
#df_ippu0[[x for x in df_ippu.columns if "glass" in x]]
df_project_ce[[x for x in df_project_ce.columns if "glass" in x]]

,emission_co2e_ch4_waso_landfilled_glass,emission_co2e_ch4_waso_open_dump_glass,qty_waso_incinerated_glass_tonne,qty_waso_landfilled_glass_tonne,qty_waso_open_dump_glass_tonne,qty_waso_recycled_glass_tonne,qty_waso_total_glass_produced_tonne
0,0.0,0.0,4810.103719,30379.602436,15189.801218,3577.897503,54210.568230
1,0.0,0.0,4622.741091,29196.259521,14598.129760,6385.305627,55045.738161
2,0.0,0.0,4421.791125,27927.101844,13963.550922,9264.386303,55809.556043
3,0.0,0.0,4208.441708,26579.631842,13289.815921,12204.932989,56504.319391
4,0.0,0.0,3988.555241,25190.875204,12595.437602,15215.197103,57199.989111
5,0.0,0.0,3760.864372,23752.827615,11876.413808,18289.214246,57877.260271
6,0.0,0.0,3527.186887,22276.969811,11138.484905,21433.677578,58561.960596
7,0.0,0.0,3286.989190,20759.931725,10379.965862,24646.494285,59246.380493
8,0.0,0.0,3040.828909,19205.235215,9602.617607,27932.707897,59941.433254
9,0.0,0.0,2788.517293,17611.688167,8805.844084,31293.495504,60646.309116


In [137]:
df_complete[[x for x in df_complete.columns if ("glass" in x) and ("frac_waso" in x)]]

,frac_waso_initial_composition_ind_glass,frac_waso_initial_composition_mun_glass,frac_waso_recycled_glass
0,0.0,0.0315,0.066
1,0.0,0.0315,0.066
2,0.0,0.0315,0.066
3,0.0,0.0315,0.066
4,0.0,0.0315,0.066
5,0.0,0.0315,0.066
6,0.0,0.0315,0.066
7,0.0,0.0315,0.066
8,0.0,0.0315,0.066
9,0.0,0.0315,0.066
